## Equity Earnings Projections
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/

### How to avoid getting blocked
- https://www.scraperapi.com/blog/5-tips-for-web-scraping 
- https://www.scrapehero.com/how-to-prevent-getting-blacklisted-while-scraping/


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [15]:
# https://hackernoon.com/scraping-yahoo-finance-data-using-python-ayu3zyl    

names=[]
prices=[]
changes=[]
percentChanges=[]
marketCaps=[]
totalVolumes=[]
circulatingSupplys=[]
 
CryptoCurrenciesUrl = "https://in.finance.yahoo.com/currencies"
r= requests.get(CryptoCurrenciesUrl)
data=r.text
soup=BeautifulSoup(data, "lxml")
 
counter = 40
for i in range(40, 404, 14):
   for listing in soup.find_all('tr', attrs={'data-reactid':i}):
      for name in listing.find_all('td', attrs={'data-reactid':i+3}):
         names.append(name.text)
      for price in listing.find_all('td', attrs={'data-reactid':i+4}):
         prices.append(price.text)
      for change in listing.find_all('td', attrs={'data-reactid':i+5}):
         changes.append(change.text)
      for percentChange in listing.find_all('td', attrs={'data-reactid':i+7}):
         percentChanges.append(percentChange.text)
pd.DataFrame({"Names": names, "Prices": prices, "Change": changes, "% Change": percentChanges})

,Names,Prices,Change,% Change
0,USD/INR,71.1790,+0.0800,+0.1125%
1,EUR/INR,78.3360,-0.2174,-0.28%
2,GBP/INR,92.5284,-0.1061,-0.11%
3,AED/INR,19.3360,+0.0220,+0.11%
4,INR/JPY,1.5391,+0.0027,+0.18%
5,SGD/INR,51.5270,-0.3740,-0.72%
6,USD/IDR,"13,714.0000",-42.0000,-0.3053%
7,USD/THB,30.9200,+0.0100,+0.0324%
8,USD/MYR,4.1195,+0.0105,+0.2555%
9,USD/ZAR,14.7559,-0.0188,-0.1272%


In [ ]:
# Dow stocks
Dow = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UTX', 'UNH', 'VZ', 'V', 'WMT', 'WBA'] 




In [3]:
# https://www.ryansmccoy.com/2019/01/28/download-eps-sales-estimates-using-python/

import pandas as pd
from datetime import datetime

df_calendar = pd.read_html(r'https://finance.yahoo.com/calendar/earnings/')

df_all_est = []

for ticker in df_calendar[0]['Symbol'].tolist():
    try:
        print(f'Downloading Estimates for {ticker}')

        df_est = pd.read_html(r'https://finance.yahoo.com/quote/{ticker}/analysis?p={ticker}')
        eps_est, rev_est = df_est[0], df_est[1]

        rev_est = rev_est.melt(id_vars=['Revenue Estimate'],var_name='Period')
        rev_est['Ticker'] = ticker
        rev_est['Item'] = "REVENUES"
        rev_est.rename(index=str, columns={"Revenue Estimate": "Statistic"}, inplace=True)
        rev_est['DateTime'] = str(datetime.utcnow())
        df_all_est.append(rev_est)

        eps_est = eps_est.melt(id_vars=['Earnings Estimate'],var_name='Period')
        eps_est['Ticker'] = ticker
        eps_est['Item'] = "EPS"
        eps_est.rename(index=str, columns={"Earnings Estimate": "Statistic"}, inplace=True)
        eps_est['DateTime'] = str(datetime.utcnow())
        df_all_est.append(eps_est)

        print(eps_est.append(rev_est).to_string())

    except Exception as e:
        print(f"Error Downloading {ticker} \n{e}\n")

df_all_merged = pd.concat(df_all_est)
df_all_merged.reset_index(drop=True, inplace=True)
df_all_merged = df_all_merged.T.reindex(['DateTime', 'Ticker', 'Item', 'Statistic', 'Period', 'value']).T

df_all_merged.to_csv(r'{}_estimates.csv'.format(datetime.today().strftime("%Y%M%d")))

ValueError: No tables found